# Homework：汽车销售聊天机器人

## 使用 GPT-4 构造汽车销售话术数据



使用 ChatGPT 构造销售数据的 Prompt ：

```
你是中国顶级的汽车销售，现在培训职场新人，请给出100条实用的销售话术。
每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [car_sale_data.txt](car_sale_data.txt) 文件中

In [36]:
with open("car_sale_data.txt", encoding="UTF-8") as f:
    car_sale_data = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [37]:
from langchain.text_splitter import CharacterTextSplitter

In [38]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [39]:
docs = text_splitter.create_documents([car_sale_data])

Created a chunk of size 115, which is longer than the specified 100
Created a chunk of size 108, which is longer than the specified 100
Created a chunk of size 105, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 110, which is longer than the specified 100
Created a chunk of size 109, which is longer than the specified 100
Created a chunk of size 109, which is longer than the specified 100
Created a chunk of size 111, which is longer tha

In [40]:
docs[0]

Document(page_content='[客户问题] 这款车的价格有点高，有没有优惠？\n[销售回答] 我理解您的顾虑。这款车的价格反映了其高品质和丰富配置。目前我们有一些特别的金融方案和礼品活动，您可以选择最适合自己的优惠方案。')

In [41]:
len(docs)

81

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [20]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [14]:
query = "这款车的价格有点高，有没有优惠？"

In [15]:
answer_list = db.similarity_search(query)

In [16]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 这款车的隔音效果好吗？
[销售回答] 这款车的隔音效果非常好，车身采用了多层隔音材料和隔音玻璃，有效减少外界噪音的干扰，为您提供一个安静舒适的驾乘环境。

[客户问题] 这款车的舒适性怎么样？
[销售回答] 这款车的舒适性非常高，座椅采用了人体工学设计，提供了很好的支撑和包裹感。此外，车内的静音效果也非常出色，给您提供了一个安静舒适的驾乘环境。

[客户问题] 这款车的自动泊车系统好用吗？
[销售回答] 这款车的自动泊车系统非常智能，只需按下按钮，系统会自动检测周围环境，并帮助您完成泊车操作。无论是平行泊车还是垂直泊车，都能轻松应对，特别适合在城市中使用。

[客户问题] 这款车的车内储物空间多吗？
[销售回答] 这款车的车内储物空间非常丰富，设计了多个储物格和杯架，可以方便地存放手机、水杯和其他小物件，提升使用的便利性。



In [17]:
db.save_local("car_sale_data")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [21]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [22]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E4696F7790>, search_kwargs={'k': 3})

In [24]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\MyWorkSpace\openai-quickstart\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

[客户问题] 我担心楼下的商业活动会很吵。
[销售回答] 我们在规划时就已经考虑到这一点，商业区和居住区有一定的距离和隔音设计。

[客户问题] 我喜欢安静，这里噪音大吗？
[销售回答] 我们特意进行了隔音设计，并且小区内部也有绿化带，整体非常安静。



In [25]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的豪宅啊？")

In [26]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 你们会提供家具吗？
[销售回答] 我们的精装房会提供基础家具和家电，让您拎包入住。

[客户问题] 都有哪些户型？
[销售回答] 我们有从一室到四室不等的多种户型，定能满足您不同的居住需求。

[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [27]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [28]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。



### 提取向量数据库中的`销售回答`

In [29]:
docs = retriever.get_relevant_documents(query)

In [30]:
docs[0].page_content

'[客户问题] 我担心楼下太吵。\n[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

In [31]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我担心楼下太吵。\n', '这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']

In [32]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [33]:
ans

'这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

#### 尝试各种问题

In [34]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [35]:
query = "我想离医院近点"

print(sales(query))

[]


D:\MyWorkSpace\openai-quickstart\venv\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [36]:
print(sales(query, 0.75))

['有的，距离我们小区不远就有几家大型综合医院。', '是的，附近有多家大型医院，医疗资源非常丰富。']


In [37]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\MyWorkSpace\openai-quickstart\venv\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



D:\MyWorkSpace\openai-quickstart\venv\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['我们有不同户型和付款方案，一定有适合您预算的。', '我们的房子位于黄金地段，升值潜力很大，转卖起来不会有问题。', '定金金额取决于您选择的房型和付款方式，我们可以详细为您解释。', '这个区域正在快速发展，未来的升值潜力非常大。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [39]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [40]:
qa_chain({"query": "你们小区有200万的房子吗？"})

D:\MyWorkSpace\openai-quickstart\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
D:\MyWorkSpace\openai-quickstart\venv\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们小区有200万的房子吗？', 'result': '我不知道。'}

In [41]:
qa_chain({"query": "小区吵不吵"})

{'query': '小区吵不吵', 'result': '这个小区特别注重居住体验，拥有良好的隔音设计，并且内部规划了绿化区域，可以有效降低噪音。'}

In [42]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


## 加载 FAISS 向量数据库已有结果

In [48]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [50]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [51]:
qa_chain({"query": "我想买别墅，你们有么"})

D:\MyWorkSpace\openai-quickstart\venv\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买别墅，你们有么', 'result': '我不知道。'}

In [52]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [53]:
qa_chain({"query": "我想买别墅，你们有么"})

D:\MyWorkSpace\openai-quickstart\venv\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '我不知道。'}

In [54]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [55]:
result = qa_chain({"query": "我想买别墅，你们有么"})

D:\MyWorkSpace\openai-quickstart\venv\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [56]:
result

{'query': '我想买别墅，你们有么', 'result': '我不知道。', 'source_documents': []}